In [14]:
import pandas as pd
import numpy as np
import os
import regex as re
import spacy as sy
import string
from urllib.parse import urlparse
from nltk.tokenize import TweetTokenizer
from deep_translator import GoogleTranslator


nlp_en = sy.load('en_core_web_sm')
all_stopwords = nlp_en.Defaults.stop_words


/Users/nitanshjain/.local/share/virtualenvs/Sem_Eval-qQJYuRaW/lib/python3.10/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.4.1) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [15]:
filename = '/Users/nitanshjain/Documents/Projects/Sem_Eval/semeval2023task3/data/ru/train-labels-subtask-1.txt'
train_sub1_df = pd.read_csv(filename, header=None, sep='\t', names=['id', 'genre'])
print(train_sub1_df.head())

      id    genre
0  24144  opinion
1  24129  opinion
2  24122  opinion
3  24124  opinion
4   2450  opinion


In [16]:
dir_name = '/Users/nitanshjain/Documents/Projects/Sem_Eval/semeval2023task3/data/ru/train-articles-subtask-1'

article_df = pd.DataFrame()

numbers = list()
headlines = list()
articles = list()

for file in os.walk(dir_name):
    for filename in file[2]:
        
        number = re.findall('[0-9]+', filename)
        numbers.append(number[0])
        x = dir_name + '/' + filename
        # print(number)
        article = ''
        with open(x) as f:
            lines = f.readlines()
            for i in range(len(lines)):
                if i==0:
                    headline = lines[0]
                    headlines.append(headline)
                elif lines[i]=="\n":
                    continue
                else:
                    # print()
                    article = article + '' + lines[i]
        articles.append(article)
            
article_df['id'] = numbers
article_df['headlines'] = headlines
article_df['articles'] = articles

article_df = article_df.astype({'id': 'int32'})

In [17]:
train_df = train_sub1_df.set_index('id').join(article_df.set_index('id'))
print(train_df.isnull().sum())
train_df

genre        0
headlines    0
articles     0
dtype: int64


,genre,headlines,articles
id,,,
24144,opinion,О крепком рубле и «голландской болезни роста» ...,Обменный курс российского рубля по отношению к...
24129,opinion,Георгий Гахария: Осуждаю буллинг послов со сто...,"Грузия, 21 июля, ГРУЗИНФОРМ. Осуждаю буллинг п..."
24122,opinion,В США раскрыли неприятную правду для Байдена\n,Американцы не верят утверждениям президента СШ...
24124,opinion,"В Крыму заявили, что воссоединение юга Украины...","Советник главы Крыма Крючков заявил, что воссо..."
2450,opinion,Андрей Союстов: Европейскими трендсеттерами ст...,России следует продолжать добиваться целей спе...
...,...,...,...
24108,reporting,«Европарламент стал жертвой фейков» - председа...,"Грузия, 10 июня, ГРУЗИНФОРМ. Вчера Европарламе..."
2471,reporting,"Спецоперация, 20 июня: российские военные унич...",Российские военные ракетой Оникс уничтожили ст...
24176,reporting,В парламенте Венгрии назвали политику антиросс...,Политика антироссийских санкций Евросоюза наск...


In [18]:
print(train_df.genre.value_counts())

opinion      93
reporting    41
satire        8
Name: genre, dtype: int64


In [19]:
def translation(y, lang, df):
    translated_val = list()
    # counta_before = 0
    # countb_before = 0
    # countc_before = 0
    # counta_after = 0
    # countb_after = 0
    # countc_after = 0
    
    
    for x in df[y]:
        try:
            if len(str(x))<5000:
                # print('If statement')
                # counta_before +=1
                translation = GoogleTranslator(source=lang, target='en').translate(x)
                translated_val.append(translation)
                # counta_after += 1
                
                
            elif len(str(x))>5000 and len(str(x))<10000:
                # translated_val.append(np.nan)
                # print('Else statement')
                # countb_before +=1
                
                split_x = [x[i:i+4999] for i in range(0, len(x), 4999)]
                translation_1 = GoogleTranslator(source=lang, target='en').translate(split_x[0])
                translation_2 = GoogleTranslator(source=lang, target='en').translate(split_x[1])
                translation = translation_1 + translation_2
                translated_val.append(translation)
                # countb_after += 1
                
                
            elif len(str(x))>10000:
                # print('Elif Statement')
                # countc_before +=1
                # translated_val.append(np.nan)
                split_x = [x[i:i+4999] for i in range(0, len(x), 4999)]
                translation_1 = GoogleTranslator(source=lang, target='en').translate(split_x[0])
                translation_2 = GoogleTranslator(source=lang, target='en').translate(split_x[1])
                translation_3 = GoogleTranslator(source=lang, target='en').translate(split_x[2])
                translation = translation_1 + translation_2 + translation_3
                translated_val.append(translation)
                # countc_after += 1
                
        except Exception as e:
            print(len(str(x)))
            # print(repr(e))
            translated_val.append(np.nan)
            
    # print(counta_before)
    # print(countb_before)
    # print(countc_before)
            
    # print(counta_after)
    # print(countb_after)
    # print(countc_after)
    
    df[y] = translated_val
    return df

In [20]:
train_trans_df = translation('headlines', 'ru', train_df)
train_trans_df = translation('articles', 'ru', train_df)

5935
5322
3855
8622
4999
3331
4400
8815
4652
8839
4099
4410
7029
13743
36228
5996
5331
6348
5183
5555
5914
15105
3592
4123
3681
4855
32997
3800
10299
7233
5792
16134
4196
13954
14164
5694
5757
6174
7672
10435
4013
19431
32575
5286
3442
3706
4644
3424
4084
6195
3450
4005
3366
6843
6016


In [21]:
print(train_trans_df.isna().sum())
train_trans_df.dropna(inplace=True)
print(train_trans_df.isna().sum())

genre         0
headlines     0
articles     55
dtype: int64
genre        0
headlines    0
articles     0
dtype: int64


In [22]:
def preprocessing(x, y, df):
    
    pos_tags_final_text = list()
    er_final_text = list()
    preprocessed_text = list()

    for x in df.loc[:,y]:

        tokenizer = TweetTokenizer()
        #tokenizing
        doc = tokenizer.tokenize(x)
        
        # removing links
        tokens = [token for token in doc if not urlparse(token).scheme]
        x = ' '.join(tokens)
        doc = nlp_en(x)
        
        # removing punctuation and white space
        tokens = [token.orth_ for token in doc if not token.is_punct | token.is_space]    
        x = ' '.join(tokens)
        
        # lower case
        x = x.lower()
        doc = nlp_en(x)

        # lemmatization
        tokens = [word.lemma_ for word in doc]   
        x = ' '.join(tokens)
        doc = nlp_en(x)  
        
        # removing punctuation and white space
        tokens = [token.orth_ for token in doc if not token.is_punct | token.is_space]    
        x = ' '.join(tokens)
        doc = nlp_en(x)
        
        # removing individual letters
        tokens = [word.text for word in doc if len(word)>=2]
        x = ' '.join(tokens)  
        # print(x)
        doc = nlp_en(x)
        
        # removing stop words
        tokens = [word for word in doc if not word in all_stopwords]
        list_of_strings  = [i.text for i in tokens]
        x = ' '.join(list_of_strings)
        doc = nlp_en(x)
        
        # Part of speech tagging
        pos_tags = [(i, i.tag_) for i in doc]
        pos_tags_final_text.append(pos_tags)
        
        # entity recognition tagging
        er =  [(i, i.label_, i.label) for i in doc.ents] 
        er_final_text.append(er)
        
        preprocessed_text.append(x)
        
    return pos_tags_final_text, er_final_text, preprocessed_text

In [23]:
pos_tags_final_headlines, er_final_headlines, preprocessed_headlines = preprocessing(headline, 'headlines', train_trans_df)

train_trans_df['preprocessed_headlines'] = preprocessed_headlines
train_trans_df['pos_tags_headlines'] = pos_tags_final_headlines
train_trans_df['er_tags_headlines'] = er_final_headlines

In [24]:
pos_tags_final_articles, er_final_articles, preprocessed_articles = preprocessing(article, 'articles', train_trans_df)

train_trans_df['preprocessed_articles'] = preprocessed_articles
train_trans_df['pos_tags_articles'] = pos_tags_final_articles
train_trans_df['er_tags_articles'] = er_final_articles

In [25]:
train_trans_df.head()

,genre,headlines,articles,preprocessed_headlines,pos_tags_headlines,er_tags_headlines,preprocessed_articles,pos_tags_articles,er_tags_articles
id,,,,,,,,,
24129,opinion,Giorgi Gakharia: I condemn the bullying of amb...,"Georgia, July 21, GEORGINFORM. I condemn the b...",giorgi gakharia condemn the bullying of ambass...,"[(giorgi, NNP), (gakharia, NNP), (condemn, VBP...","[((georgia), GPE, 384)]",georgia july 21 georginform condemn the bullyi...,"[(georgia, NNP), (july, NNP), (21, CD), (georg...","[((georgia), GPE, 384), ((july, 21), DATE, 391..."
24122,opinion,In the United States revealed the unpleasant t...,Americans do not believe US President Joe Bide...,in the united states reveal the unpleasant tru...,"[(in, IN), (the, DT), (united, NNP), (states, ...","[((the, united, states), GPE, 384)]",americans do not believe us president joe bide...,"[(americans, NNPS), (do, VBP), (not, RB), (bel...","[((americans), NORP, 381), ((joe, biden), PERS..."
24124,opinion,"In Crimea, they said that the reunification of...",Adviser to the head of the Crimea Kryuchkov sa...,in crimea they say that the reunification of t...,"[(in, IN), (crimea, NNP), (they, PRP), (say, V...","[((ukraine), GPE, 384), ((russia), GPE, 384)]",adviser to the head of the crimea kryuchkov sa...,"[(adviser, NN), (to, IN), (the, DT), (head, NN...","[((ukraine), GPE, 384), ((russia), GPE, 384), ..."
2414,opinion,"The West received the command ""face"": Lavrov ""...",The head of the Russian Ministry of Foreign Af...,the west receive the command face lavrov tear ...,"[(the, DT), (west, NNP), (receive, VBP), (the,...",[],the head of the russian ministry of foreign af...,"[(the, DT), (head, NN), (of, IN), (the, DT), (...","[((the, russian, ministry, of, foreign, affair..."
242,opinion,"""Merkel was smarter!"" - Scholz on the supply o...","On the issue of arms supplies to Ukraine, Mosc...",merkel be smart scholz on the supply of weapon...,"[(merkel, NNP), (be, VB), (smart, JJ), (scholz...","[((germany), GPE, 384)]",on the issue of arm supply to ukraine moscow l...,"[(on, IN), (the, DT), (issue, NN), (of, IN), (...","[((moscow), GPE, 384), ((germany), GPE, 384), ..."


In [26]:
# converting tweets_df into a csv file
filename = '/Users/nitanshjain/Documents/Projects/Sem_Eval/semeval2023task3/preprocessed_data/subtask1/ru_train_subtask_1.csv'
train_trans_df.to_csv(filename, index=True)